# Mask R-CNN Species Photogrammetry

### Environment Setup

In [3]:
import os
import sys
import random
import cv2
import math
import re
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imgaug
from imgaug import augmenters as iaa
import numpy.ma as ma
import scipy.misc
import skimage.filters

# Root directory of the project
MASK_RCNN_DIR = os.path.abspath("../Mask_RCNN/")
ROOT_DIR = os.path.abspath(".")

# Import Mask RCNN
sys.path.append(MASK_RCNN_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import whale

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


Using TensorFlow backend.


### Model and dataset setup

In [4]:
config = whale.WhaleConfig()
WHALE_DIR = os.path.abspath("../photogram_data/")

# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
# Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0" # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"




In [5]:
# Load dataset to measure
dataset = whale.WhaleDataset()
dataset.load_whale(WHALE_DIR, "test")
#dataset.load_whale(WHALE_DIR, "test")

# Must call before using the dataset
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 62
Classes: ['BG', 'balaenoptera_musculus_body', 'balaenoptera_musculus_pectoral', 'megaptera_novaeangliae_body', 'megaptera_novaeangliae_pectoral', 'balaenoptera_acutorostrata_body', 'balaenoptera_acutorostrata_pectoral']


In [6]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [7]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                          config=config)

## Function Definitions

In [8]:
def run_cnn(weights_path="logs/whale20181014T1536/mask_rcnn_whale_0174.h5", verbose=0):
        
    # Load weights
    print("Loading weights ", weights_path)
    model.load_weights(weights_path, by_name=True)
    print("Weights loaded.")
    
    
    detection_results = []
    initial_time = time.perf_counter()
    for image_id in dataset.image_ids:
    
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

        info = dataset.image_info[image_id]

        # Run object detection
        results = model.detect([image], verbose=0)
        
        #detection_results.append([results, info, image_id])
        detection_results.append([results, info['id'], image_id, {'class_ids' : gt_class_id, 'masks' : gt_mask, 'rois' : gt_bbox}])
        if verbose:
            print("Done detecting and masking image #:  " + str(len(detection_results)))
    
    finish_time = time.perf_counter()
    time_elapsed = finish_time - initial_time
    if verbose:
        print("\n")
        print(time_elapsed, "seconds elapsed while masking", len(detection_results), "images.")
        print((time_elapsed/len(detection_results), "seconds per image."))
    
    return detection_results



In [9]:
def pca(body, visualize=False):
    # based on https://alyssaq.github.io/2015/computing-the-axes-or-orientation-of-a-blob/
    # and based on http://alyssaq.github.io/2015/visualising-matrices-and-affine-transformations-with-python/#rotating
    xy_array = []
    # get indexes of mask pixels
    y, x = np.nonzero(body)
    
    # mean center the coords
    x = x - np.mean(x)
    y = y - np.mean(y)
    coords = np.vstack([x, y])

    # build covariance matreix and eigenvectors
    cov = np.cov(coords)
    evals, evecs = np.linalg.eig(cov)
    
    # sort eigenvalues
    sort_indices = np.argsort(evals)[::-1]
    x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
    x_v2, y_v2 = evecs[:, sort_indices[1]]

    
    if visualize is True:
        # plot the major and minor axis of the whale mask
        scale = 20
        plt.plot(x, y, 'k.')

        plt.plot([x_v1*-scale*2, x_v1*scale*2],
         [y_v1*-scale*2, y_v1*scale*2], color='red')
        plt.plot([x_v2*-scale, x_v2*scale],
         [y_v2*-scale, y_v2*scale], color='blue')
        plt.axis('equal')
        plt.gca().invert_yaxis()  # Match the image system with origin at top left
        plt.show()

    # orient this along the horizontal axis
    theta = np.tanh((x_v2)/(y_v2))  
    
    # TODO this is a hack, for some reason this doesn't work when theta is high or aka when the actual angle is small
    if abs(theta) > 0.9:
        theta = np.tanh((x_v1)/(y_v1))
        theta = theta + 0.5 *math.pi
    rotation_mat = np.matrix([[np.cos(theta), -np.sin(theta)],
                      [np.sin(theta), np.cos(theta)]])
    transformed_mat = rotation_mat * coords
    
    # plot the transformed blob
    # these are the final transformed coords
    x_transformed, y_transformed = transformed_mat.A

    maxX = np.max(x_transformed)
    minX = np.min(x_transformed)
    maxY = np.max(y_transformed)
    minY = np.min(y_transformed)


    # Get corresonding Y values for minX and maxX
    maxX_index = np.where(x_transformed == maxX)  # index of right-most point
    rightY = float((y_transformed[maxX_index]))   # corresponding Y value


    minX_index = np.where(x_transformed == minX)  # index of left-most point
    leftY = float((y_transformed[minX_index]))    # corresponding Y value
    
    # Orient the mask correctly - flip so the fluke is on the right

    # Get corresonding X values for maxY and minY

    maxY_index = np.where(y_transformed == maxY) #index of top point
    topX = float((x_transformed[maxY_index])) #corresponding X value


    minY_index = np.where(y_transformed == minY) #index of bottom point
    bottomX = float((x_transformed[minY_index])) #corresponding X value

    # flip mask so fluke is on the right, if necessary
    if (topX < 0 or bottomX < 0):
        x_transformed = x_transformed*-1 
    
    xy_array = [x_transformed, y_transformed]
    
    
    return xy_array


In [10]:
def measure(body_mask, visualize=False):
    xy_array = pca(body_mask, visualize=visualize)
    x_transformed = xy_array[0]
    y_transformed = xy_array[1]

    # Reassign max/min X values in case image was flipped during PCA
    maxX = np.max(x_transformed) # Right-most point
    minX = np.min(x_transformed) # Left-most point

    # Get corresponding Y values for maxX and minX
    maxX_index = np.where(x_transformed == maxX)  # index of right-most point
    rightY = float((y_transformed[maxX_index]))   # corresponding Y value

    minX_index = np.where(x_transformed == minX)  # index of left-most point
    leftY = float((y_transformed[minX_index]))    # corresponding Y value

    # TODO come up with a better solution here

    # Draw a straight line across the mask

    # Filter out points close to the midline of the mask (on the Y axis)
    # TODO arbitrary lambda, might need to change later
    lowEnough = list(filter(lambda y: y < (leftY + 0.5), y_transformed)) #above midline
    yValues = list(filter(lambda y: y > (leftY - 0.5), lowEnough)) #below midline
    yValues.sort()

    # Get corresponding X values to draw the line

    # List of appropriate indices
    indices = []
    for point in yValues:
        index = int(np.where(y_transformed == point)[0])
        indices.append(index)

    xValues = [] # Corresponding X values
    for index in indices:
        xValues.append(x_transformed[index]) 

    xValues.sort()

    # Use distance formula to measure the length from the midline
    length = math.sqrt((xValues[-1] - xValues[0])**2 + (yValues[-1] - yValues[0])**2)
    
    return(length)
        

In [11]:
# Each image could have multiple masks (body and pectoral) and multiple animals
# Return all of the body masks

def find_correct_masks(mask_list): 
    class_body_array = []
    for index, class_id in enumerate(mask_list['class_ids']):
        if class_id % 2 != 0: # all body class ids are odd numbers
            if 'scores' in mask_list:
                class_body_array.append([class_id, mask_list['masks'][:,:,index], mask_list['scores'][index]])
            else:
                class_body_array.append([class_id, mask_list['masks'][:,:,index]])
    return(class_body_array)

In [12]:
def append_measurements(results_list, manual=False):
    for detection_result in results_list:
        detected_mask_list = detection_result[0][0]
        gt_mask_list = detection_result[3]
            
        detected_body_list = find_correct_masks(detected_mask_list)  
        gt_body_list = find_correct_masks(gt_mask_list)

        detected_body_lengths = []
        for class_id_body in detected_body_list:
            # adding class id, body length, and scores from CNN
            detected_body_lengths.append([class_id_body[0], measure(class_id_body[1]), class_id_body[2]])
        
        gt_body_lengths = []
        for class_id_body in gt_body_list:
            gt_body_lengths.append([class_id_body[0], measure(class_id_body[1])])
       
        detection_result.append(detected_body_lengths)
        detection_result.append(gt_body_lengths)
        # detection_results is now [results, info['id'], image_id, gt_mask_dict, [class_id, cnn_lengths, cnn_scores], [class_id, gt_lengths]]
            
    return(True)
    

In [14]:
def get_pd_from_csv(csv_fn):
    measurements = pd.read_csv(csv_fn)
        
    blue_measurements = measurements.loc[measurements["Whale"].str.contains("Bm")] 
    blue_measurements = blue_measurements.reset_index(drop=True)
    
    humpback_measurements = measurements.loc[measurements["Whale"].str.contains("Mn")] 
    humpback_measurements = humpback_measurements.reset_index(drop=True)
    
    minke_measurements = measurements.loc[measurements["Whale"].str.contains("Bb")] 
    minke_measurements = minke_measurements.reset_index(drop=True)

    return(blue_measurements, humpback_measurements, minke_measurements)

In [15]:
def pixel_to_meters(pixels, size_factor, pixel_size, focal_length, total_altitude):
    adjusted_pixel_count = pixels * size_factor
    return(adjusted_pixel_count * pixel_size/focal_length * total_altitude)
    

def convert_measurements(merged_df, org_img_size=(6000.0, 4000.0)):
    all_lengths = []

    # imaged were downsized from 6000 as a max dimension to 1024 as a max dimension so pixels are 5x as large in meters
    pixel_size_factor =  org_img_size[0] / 1024.0
    
    for i, row in merged_df.iterrows():
        det_length = pixel_to_meters(row['detected_pix_len'], pixel_size_factor, row['Pixel size'], row["Focal length (mm)"], row["Total Altitude"])   
        merged_df.at[i,'detected_len'] = det_length
        
        gt_length = pixel_to_meters(row['gt_pix_len'], pixel_size_factor, row['Pixel size'], row["Focal length (mm)"], row["Total Altitude"])   
        merged_df.at[i,'gt_len'] = gt_length
    
    return(True)

In [16]:
def create_df(appended_detections, measurements_csv_fn):
    detections_df = pd.DataFrame(columns=['Image','detected_class','detected_pix_len','gt_class','gt_pix_len', 'detection_score', 'IoU'])
    for index, img in enumerate(appended_detections):
        try:
            longest_length = 0
            longest_mask_index = None
            longest_detection = None
            for mask_index, detection in enumerate(img[4]): # this is the detected mask class and can have multiple
                # take the longest mask
                if detection[1] > longest_length:
                    longest_detection = detection
                    longest_length = detection[1]
                    longest_mask_index = mask_index
            
            img.append([longest_mask_index, longest_length])
            
            gt_longest_length = 0
            gt_longest_mask_index = None
            gt_longest_detection = None
            for mask_index, detection in enumerate(img[5]): # this is the gt mask class and can have multiple
                # take the longest mask
                if detection[1] > gt_longest_length:
                    gt_longest_detection = detection
                    gt_longest_length = detection[1]
                    gt_longest_mask_index = mask_index

            #print(img[0][0]['masks'][longest_detection])
            
            #"""
            ap = utils.compute_ap_range(np.array([img[3]['rois'][gt_longest_mask_index]]), 
                                        np.array([img[3]['class_ids'][gt_longest_mask_index]]), 
                                        np.array([img[3]['masks'][gt_longest_mask_index]]),
                                        np.array([img[0][0]['rois'][longest_mask_index]]), 
                                        np.array([img[0][0]['class_ids'][longest_mask_index]]), 
                                        np.array([img[0][0]['scores'][longest_mask_index]]), 
                                        np.array([img[0][0]['masks'][longest_mask_index]]), 
                                        iou_thresholds=None)
                        
            iou_array = utils.compute_overlaps_masks(img[3]['masks'], img[0][0]['masks'])
            iou = iou_array.max()
            #print('IOU is: ', iou)
            
            detections_df = detections_df.append({"Image": img[1], "detected_class": longest_detection[0], 
                                "detected_pix_len": longest_detection[1], 'gt_class' : img[5][0][0],
                                "gt_pix_len" : img[5][0][1], 'detection_score' : longest_detection[2], 'IoU' : iou},
                                ignore_index=True)
        except IndexError: # in case a detection was not made
            print("Index error")
            pass
        except TypeError: # also in case a detection was not made
            print("TypeError")
            pass
        
    measurements = pd.read_csv(measurements_csv_fn)
    
    merged_df = detections_df.merge(measurements,on='Image', how='inner') #.dropna(subset=['id'])
    
    return(merged_df)
        
    

In [17]:
def show_example(body_mask, visualize=False):
    xy_array = pca(body_mask, visualize=visualize)
    x_transformed = xy_array[0]
    y_transformed = xy_array[1]

    # Reassign max/min X values in case image was flipped during PCA
    maxX = np.max(x_transformed) # Right-most point
    minX = np.min(x_transformed) # Left-most point

    # Get corresponding Y values for maxX and minX
    maxX_index = np.where(x_transformed == maxX)  # index of right-most point
    rightY = float((y_transformed[maxX_index]))   # corresponding Y value

    minX_index = np.where(x_transformed == minX)  # index of left-most point
    leftY = float((y_transformed[minX_index]))    # corresponding Y value

    # TODO come up with a better solution here

    # Draw a straight line across the mask

    # Filter out points close to the midline of the mask (on the Y axis)
    # TODO arbitrary lambda, might need to change later
    lowEnough = list(filter(lambda y: y < (leftY + 0.5), y_transformed)) #above midline
    yValues = list(filter(lambda y: y > (leftY - 0.5), lowEnough)) #below midline
    yValues.sort()

    # Get corresponding X values to draw the line

    # List of appropriate indices
    indices = []
    for point in yValues:
        index = int(np.where(y_transformed == point)[0])
        indices.append(index)

    xValues = [] # Corresponding X values
    for index in indices:
        xValues.append(x_transformed[index]) 

    xValues.sort()

    # Use distance formula to measure the length from the midline
    length = math.sqrt((xValues[-1] - xValues[0])**2 + (yValues[-1] - yValues[0])**2)
    
    if visualize:    
        #plt.plot(x_transformed, y_transformed, alpha = 0.5)
        plt.plot(x_transformed, y_transformed, 'g.', zorder=0)

        # set axis limits
        plt.xlim([minX - 30, maxX + 30])
        plt.ylim([leftY - 175, rightY + 175])

        #Plot the first and last points from the list, use this for length

        #plt.scatter(xValues[0], yValues[0], zorder=10)
        #plt.scatter(xValues[-1], yValues[-1], zorder=10)
        plt.scatter(xValues, yValues, zorder=10)
        plt.show()
    
    return(length)
        

## Function Calls

In [92]:
detection_results = run_cnn(weights_path = 'logs/whale20181014T1536/mask_rcnn_whale_0174.h5')

Loading weights  /home/clifgray/Code/cetacean_photogram/logs/whale20181014T1536/mask_rcnn_whale_0174.h5
Re-starting from epoch 174
Weights loaded.


In [93]:
append_measurements(detection_results) # CNN appends length 

True

In [ ]:
detection_df = create_df(detection_results, '../photogram_data/whale_measurements.csv')

In [95]:
convert_measurements(detection_df)

True

### How did we do?

In [ ]:
plt.scatter(detection_df['Total Length (m)'], detection_df['detected_len'])